In [35]:
import os
import pandas as pd
import pandas as pd
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon

Statistical significance analysis
==

The objective of this script is to evaluate the significance of observed difference in term of energy consumption. 

### Scenario wise comparison

Compare the distribution of each utility scenarios to a baseline (bloated or GNU depending the RQ). Each scenario is executed 10 times.

In [36]:
current_dir = os.getcwd()

def computeMannWhitneyU(input_file_path, utilities, baseline):
   
    df = pd.read_csv(input_file_path, delimiter=";", engine="python")
    results = []
    grouped = df.groupby(["Program", "TestNr"])

    for (program, test_nr), group in grouped:
        psys_bloated = group[group["Utilities"] == baseline]["PSYS"]

        for utility in utilities:
            # Utility energy consumption
            psys_other = group[group["Utilities"] == utility]["PSYS"]
            if len(psys_bloated) > 0 and len(psys_other) > 0:
                u_stat, p_value = mannwhitneyu(psys_other, psys_bloated, alternative="two-sided")
                results.append((program, test_nr, utility, u_stat, p_value))

    results_df = pd.DataFrame(results, columns=["Program", "TestNr", "Utility", "U_stat", "p_value"])

    return results_df


def check_significance(results_df, stat="U_stat"):
    """
    Groups the results by (Utility, Program) and checks:
    - If all p-values in the group are significant (p < 0.05).
    - Whether the utility reduces or increases PSYS based on U-statistic.

    Parameters:
        results_df (pd.DataFrame): DataFrame with columns ["Program", "TestNr", "Utility", "U_stat", "p_value"]

    Returns:
        pd.DataFrame: DataFrame with columns ["Program", "Utility", "All_Significant", "Effect"]
    """
    # Group by (Program, Utility) and check if all p-values are < 0.05
    summary_df = (
        results_df
        .groupby(["Program", "Utility"])
        .agg(
            All_Significant=("p_value", lambda p: (p < 0.05).all()),  # All tests significant?
            Mean_U_stat=(stat, "mean")  # Compute average U-statistic
        )
        .reset_index()
    )

    # Determine the effect using U-statistic
    summary_df["Effect"] = summary_df.apply(
        lambda row: "Lower" if row["Mean_U_stat"] < (10 * 10) / 2 else "Higher", axis=1
    )

    return summary_df




#### RQ1 - Debloated vs. bloated

**Research hypothesis:** debloating techniques , impact the energy consumption  

**Null hypothesis:** Debloated utility [DEBOP/COV/CHISEL] have the same distribution in term of energy consumption compared to the bloated utility [BLOATED]  

In [59]:
results_dir = os.path.join(current_dir, "debloating_results")
debloat_result = os.path.join(results_dir, 'cat_all_repeats.csv')

utilities_to_compare = ["chisel", "debop", "cov"]
baseline = "bloated"
df =computeMannWhitneyU(debloat_result,utilities_to_compare, baseline)
df

,Program,TestNr,Utility,U_stat,p_value
0,date,1,chisel,43.0,0.623176
1,date,1,debop,40.0,0.472509
2,date,1,cov,39.0,0.427355
3,date,2,chisel,53.0,0.850107
4,date,2,debop,53.0,0.850107
5,date,2,cov,57.0,0.623176
6,grep,1,chisel,100.0,0.000182
7,grep,1,debop,44.0,0.677585
8,grep,1,cov,43.0,0.623176
9,grep,2,chisel,100.0,0.000183


In [60]:
sig =check_significance(df)
significan_utility = sig[sig["All_Significant"] == True]
significan_utility.to_csv("debloating_results/significant_utility.csv", index=False)

In [61]:
percentage_significant = 100- len(significan_utility) / len(sig) * 100
print(f" {percentage_significant} % of the results are significant")

 88.88888888888889 % of the results are significant


#### RQ1 - ALTERNATIVE 

In [54]:
results_dir = os.path.join(current_dir, "results")
alternative_result = os.path.join(results_dir, 'cat_all_repeats.csv')
utilities_to_compare = ["ToyBox", "BusyBox"]
baseline = "GNU"
df=computeMannWhitneyU(alternative_result,utilities_to_compare, baseline)
df

,Program,TestNr,Utility,U_stat,p_value
0,base64,1,ToyBox,100.0,0.000183
1,base64,1,BusyBox,92.0,0.001706
2,base64,2,ToyBox,100.0,0.000183
3,base64,2,BusyBox,100.0,0.000183
4,basename,1,ToyBox,44.5,0.705351
...,...,...,...,...,...
105,true,2,BusyBox,46.0,0.791337
106,wc,1,ToyBox,0.0,0.000183
107,wc,1,BusyBox,0.0,0.000183
108,wc,2,ToyBox,0.0,0.000183


In [55]:
sig =check_significance(df)
significan_utility = sig[sig["All_Significant"] == True]
significan_utility.to_csv("results/significant_utility.csv", index=False)

In [56]:
significan_utility

,Program,Utility,All_Significant,Mean_U_stat,Effect
0,base64,BusyBox,True,96.0,Higher
1,base64,ToyBox,True,100.0,Higher
4,cat,BusyBox,True,100.0,Higher
5,cat,ToyBox,True,100.0,Higher
8,cksum,BusyBox,True,96.5,Higher
9,cksum,ToyBox,True,97.0,Higher
10,comm,BusyBox,True,100.0,Higher
11,comm,ToyBox,True,97.0,Higher
14,cut,BusyBox,True,100.0,Higher
15,cut,ToyBox,True,100.0,Higher


In [57]:
percentage_significant = 100 - len(significan_utility) / len(sig) * 100
print(f" {percentage_significant} % of the results are not significant")

 67.85714285714286 % of the results are not significant


## Analysis at global level

In [43]:
def computeMannWhitneyUGlobal(input_file_path, utilities, baseline):
    """
    Performs the Mann-Whitney U test on PSYS values between a baseline utility 
    and each specified utility across all measurements.
    
    Parameters:
        input_file_path (str): Path to the CSV file.
        utilities (list): List of utilities to compare against the baseline.
        baseline (str): The reference utility (e.g., "bloated").
    
    Returns:
        pd.DataFrame: DataFrame with columns ["Utility", "U_stat", "p_value"].
    """
    # Load CSV file
    df = pd.read_csv(input_file_path, delimiter=";", engine="python")

    # Extract PSYS values for the baseline utility
    psys_bloated = df[df["Utilities"] == baseline]["PSYS"]

    # Initialize results list
    results = []

    for utility in utilities:
        if utility == baseline:
            continue  # Skip comparing the baseline to itself

        # Extract PSYS values for the current utility
        psys_other = df[df["Utilities"] == utility]["PSYS"]

        # Ensure both samples have data before performing the test
        if len(psys_bloated) > 0 and len(psys_other) > 0:
            # Perform Mann-Whitney U test
            u_stat, p_value = mannwhitneyu(psys_other, psys_bloated, alternative="two-sided")

            # Store results
            results.append((utility, u_stat, p_value))

    # Convert results into a DataFrame
    results_df = pd.DataFrame(results, columns=["Utility", "U_stat", "p_value"])

    return results_df
    

    

In [44]:
results_dir = os.path.join(current_dir, "debloating_results")
debloat_result = os.path.join(results_dir, 'cat_all_repeats.csv')

utilities_to_compare = ["chisel", "debop", "cov"]
baseline = "bloated"
df =computeMannWhitneyUGlobal(debloat_result,utilities_to_compare, baseline)
df

,Utility,U_stat,p_value
0,chisel,8001.5,0.136358
1,debop,6890.5,0.565565
2,cov,7034.5,0.758976


In [45]:
results_dir = os.path.join(current_dir, "results")
alternative_result = os.path.join(results_dir, 'cat_all_repeats.csv')
utilities_to_compare = ["ToyBox", "BusyBox"]
baseline = "GNU"
df=computeMannWhitneyUGlobal(alternative_result,utilities_to_compare, baseline)
df

,Utility,U_stat,p_value
0,ToyBox,152893.0,0.755212
1,BusyBox,143035.5,0.118960
